<div style="font-size: 200%; font-weight: bold; color: maroon;">Propagación de una infección en una red de contactos</div>
<!-- Paulo Villegas, 2021 - v. 2.0 -->

Vamos a simular la propoagación de una enfermedad hipotética usando la red de contactos escolares.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
from itertools import chain

%matplotlib inline

In [ ]:
spark

In [ ]:
import pyspark.sql
import pyspark.sql.functions as F

# Leer datos

Regeneramos la red a partir de los dataframes guardados.

In [ ]:
import graphframes as gf

nodes = spark.read.parquet('OUT/contact-network-vertices')
edges = spark.read.parquet('OUT/contact-network-edges')

cn = gf.GraphFrame(nodes, edges)

In [ ]:
cn.vertices.count(), cn.edges.count()

In [ ]:
edges.printSchema()

# Propagación de la infección

Realizamos la propagación a través del grafo monitorizando los tiempos de cada contacto y comparando con el tiempo de infección de cada nodo. Por simplificar, vamos a suponer una tasa de contagio del 100%: cada vez que un nodo entra en contacto con un nodo infectado, se contagia.

## Subgrafo

El grafo original tiene una conectividad tan alta que no es muy interesante para observar propagación: enseguida se propaga por completo. En su lugar vamos a estudiar un subgrafo mucho más pequeño, formado **solo por un 1%** de los contactos originales

In [ ]:
edges_prop = edges.sample(0.01, seed=12345)
edges_prop.count()

In [ ]:
edges_prop.groupby(F.date_trunc('day', 'time').alias('day')).count().sort('day').toPandas()

In [ ]:
gprop = gf.GraphFrame(nodes, edges_prop)

In [ ]:
d = gprop.degrees.join(nodes, 'id').sort('degree', ascending=True)

In [ ]:
d.groupby('degree').count().toPandas()

In [ ]:
d.filter(d.degree==3).show()

## Punto de entrada de la infección

Creamos una columna que marque el momento en que una persona ha sido nominalmente "infectado", por haber estado en contacto con otra que estaba infectada.

La inicializamos a una fecha en un futuro posterior al intervalo de los contactos, para marcar que los individuos inicialmente están sanos.

In [ ]:
import datetime

# Una fecha arbitraria en el futuro
healthy = int(datetime.datetime(year=3000, month=1, day=1).timestamp())

Marcamos el "**paciente 0**": paciente infectado que entra en la red de contactos

In [ ]:
patient0 = 1894

# Marcamos al paciente 0
nodes_prop = nodes.withColumn('infect_ts', F.when(nodes.id == patient0, F.lit(0)).otherwise(F.lit(healthy)))

In [ ]:
# Comprobemos que efectivamente tenemos una sola persona infectada
nodes_prop.filter(nodes_prop.infect_ts != healthy).count()

Veamos los contactos del paciente 0

In [ ]:
def contact_list(g: gf.GraphFrame, node_id: int) -> pyspark.sql.DataFrame:
    '''
    Return the list of contacts made by one node
    '''
    s = g.edges.filter((g.edges.src == node_id) | (g.edges.dst == node_id))
    s = s.select(F.when(s.src == node_id, s.dst).otherwise(s.src).alias('node'), s.time)
    return s.sort(['node', 'time'])

In [ ]:
contact_list(gprop, patient0).show()

## Iteración

Realizamos ahora la iteración de `AggregateMessages`. En ella vamos a propagar las infecciones entre nodos, usando como filtro la marca de tiempo del contacto (enlace): una infección solo puede propagarse por un enlace _si es anterior a la marca de tiempo del enlace_. 

La siguiente figura ilustra el proceso, mostrando como ejemplo un nodo $I_{\infty}$ que tiene 6 enlaces. Se muestran los 6 mensajes que llegarían $I_{\infty}$ ; uno de ellos (el marcado en rojo) es el que propagaría la infección hasta él.

![Propagación](img/aggregate_messages.png)

Nótese que en cada iteración efectuamos el paso de mensajes (iguales) _en los dos sentidos_, ya que la propagación puede hacerse en las dos direcciones del contacto; es indiferente cuál es el nodo origen y cual el destino del grafo.

In [ ]:
from graphframes.lib import AggregateMessages as AM

def propagate(nodes: pyspark.sql.DataFrame, edges: pyspark.sql.DataFrame) -> gf.GraphFrame:

    # Creamos el grafo de propagación
    gprop = gf.GraphFrame(nodes, edges)

    # Propagación src->dst: si el nodo "src" está infectado en el momento del contacto, 
    # marca al nodo "dst" como infectado en ese momento, si no lo estaba ya
    msgToDst = F.when((AM.src.infect_ts < AM.edge.ts) & 
                      (AM.edge.ts < AM.dst.infect_ts), AM.edge.ts).otherwise(AM.dst.infect_ts)

    # Propagación dst->src: lo mismo, intercambiando los nodos
    msgToSrc = F.when((AM.dst.infect_ts < AM.edge.ts) & 
                      (AM.edge.ts < AM.src.infect_ts), AM.edge.ts).otherwise(AM.src.infect_ts)

    # Itera hasta que no se infecta nadie más    
    infected_prev = 0
    for iter in range(1, 20):

        # Agrega los mensajes enviados entre nodos
        agg = gprop.aggregateMessages(F.min(AM.msg).alias('infect_ts'),
                                      sendToSrc=msgToSrc, sendToDst=msgToDst)
        v = gprop.vertices
        v = v.drop('infect_ts').join(agg, on='id', how='inner')

       # Comprueba la condición de parada
        infected_curr = v.filter(v.infect_ts != healthy).count()
        print('Iteration: {:2} => infected: {:3}'.format(iter, infected_curr))
        if infected_curr == infected_prev:
            break
        infected_prev = infected_curr

        # Prepara el grafo para ls siguiente iteración
        cached_v = AM.getCachedDataFrame(v)
        gprop = gf.GraphFrame(cached_v, gprop.edges)
        
    return gprop

Realizamos la iteración

In [ ]:
%%time

gprop = propagate(nodes_prop, edges_prop)

# Resultados


In [ ]:
def node_infection(g: gf.GraphFrame) -> pyspark.sql.DataFrame:
    '''
    Devuelve un DataFrame de nodos con información de propagación: una columna que marca su momento de infección, 
    en formato de fecha
    '''
    # Convertimos la marca de tiempo en una fecha representable, excepto para los nodos no infectados
    final = g.vertices
    final = final.withColumn('infect_time', 
                             F.when(final.infect_ts < F.lit(healthy), F.from_unixtime(final.infect_ts)))
    # Devolvemos la lista en orden de infección
    return final.sort('infect_ts')

In [ ]:
infection_list = node_infection(gprop)

In [ ]:
list_size = infection_list.count()
list_size

## Lista de personas sanas

Obtenemos la lista de las personas que, después de haber realizado todas las iteraciones de propagación, no han sido infectadas

In [ ]:
healthy_persons = infection_list.filter(infection_list['infect_time'].isNull())

In [ ]:
num_healthy = healthy_persons.count()
num_healthy

In [ ]:
healthy_persons.toPandas()

## Lista de infectados (en orden de infección)

In [ ]:
num_infected = list_size - num_healthy
num_infected

In [ ]:
# Lista de infectados, en orden
d = infection_list.limit(num_infected+1).toPandas()

with pd.option_context('max_rows', num_infected+1):
    display(d)

## Rastreo de contactos: red ego del paciente 0

Vamos a hacer una pequeña prueba de _contact tracing_: Calculamos la red ego ampliada del paciente 0: hasta dos contactos a partir de él

In [ ]:
def n_hops(g: gf.GraphFrame, node_id: int, hops: int=1) -> pyspark.sql.DataFrame:
    '''
    Return all edges starting from a node up to N hops
    '''
    origin = {node_id}
    prev = set()
    res = None
    for n in range(hops):
        s1 = g.edges.filter(g.edges.src.isin(origin))
        s2 = g.edges.filter(g.edges.dst.isin(origin))
        s = s1.union(s2)
        nodes = chain.from_iterable((r.src, r.dst) for r in s.select('src', 'dst').collect())
        origin = set(nodes) - prev
        prev |= origin
        res = s if not res else res.union(s)
    res = res.distinct()
    return res.sort(['src', 'dst', 'time'])

In [ ]:
p0_ego = n_hops(gprop, patient0, 2)

Añadimos la fecha de infección de cada nodo de la red

In [ ]:
infect_times = infection_list.select('id', 'infect_time')

# Añadimos la fecha de infección del nodos src
ego = p0_ego.join(infect_times, infect_times.id == p0_ego.src) \
            .withColumnRenamed('infect_time','time_src') \
            .drop('id')

# Añadimos la fecha de infección del nodos dst
ego = ego.join(infect_times, infect_times.id == p0_ego.dst) \
            .withColumnRenamed('infect_time','time_dst') \
            .drop('id')


Creamos una función para representar un grafp

In [ ]:
from math import sqrt
from collections import defaultdict

def draw_subgraph(df: pd.DataFrame, mark=None):
    '''
    Given a DataFrame of edges, draw the corresponding graph.
    Optionally, highlight a given node
    '''
    ## Create NetworkX graph
    gnx = nx.from_pandas_edgelist(df,'src','dst', ['time'], create_using=nx.MultiGraph())
    
    # Prepare the labels for each edge: first contact time *after* infection
    lbl = defaultdict(list)
    for r in df.itertuples():
        if (r.time_src and r.time > r.time_src) or (r.time_dst and r.time > r.time_dst):
            lbl[(r.src, r.dst)].append(r.time)
    lbl = {k: '\n'.join(min(v).split()) for k, v in lbl.items()}
    
    # Node color
    if mark:
        color = ['lightcoral' if v == mark else 'yellow' for v in gnx.nodes()]
    else:
        color = 'yellow'
    
    # Graph positions
    pos = nx.spring_layout(gnx, k=3/sqrt(len(gnx)), seed=1143)

    # Draw
    fig, ax = plt.subplots(1, 1, figsize=(16, 10))
    nx.draw_networkx_nodes(gnx, pos, ax=ax, node_size=400, node_color=color)
    nx.draw_networkx_labels(gnx, pos, ax=ax)
    nx.draw_networkx_edges(gnx, pos, ax=ax, edge_color='lightblue');
    nx.draw_networkx_edge_labels(gnx, pos, ax=ax, edge_labels=lbl, font_size=9);

Representamos el grafo alrededor del paciente 0, resaltándolo

In [ ]:
draw_subgraph(ego.toPandas(), patient0)

# Ejercicio

Repetir el proceso, pero esta vez suponiendo que el paciente 0 se infectó el 4 de diciembre a las 0h (y **solo** a partir de ese momento es susceptible de infectar a contactos)

<div class="alert alert-warning" role="alert" style="margin-top: 24pt; margin-bottom: 0pt;">Escriba las líneas necesarias

Nota: puede usar las funciones definidas arriba</div>